In [100]:
from sklearn.cluster import KMeans
import import_ipynb
from news_classes import *
import math
from textblob import TextBlob as tb
import os
import pickle
import numpy as np

In [101]:
idf_path='bdnews_idfblob_25_18.pkl'
with open(idf_path, 'rb') as input:
    idfblob = pickle.load(input)

In [102]:
def tf(word, lower_sentences):
    ret=0
    tot=1
    for sentence in lower_sentences:
        ret+=(sentence.words.count(word))
        tot+=len(sentence.words)
    return ret / tot

In [103]:
def idf(word):
    return math.log(idfblob.size() / (1 + idfblob.getIdf(word)))

In [104]:
def tfidf(word, lower_sentences):
    return tf(word, lower_sentences) * idf(word)

In [105]:
def sentence_tfidf(sentence, lower_sentences):
    tot=0;
    for word in sentence.words:
        tot+=tfidf(word,lower_sentences)
    try:
        return tot/len(sentence.words)
    except:
        return 0

In [106]:
def get_sent_idfs(lower_sentences):
        
    sent_tfidf=[]
    for sentence in lower_sentences:
        sent_tfidf.append(sentence_tfidf(sentence, lower_sentences))

    return sent_tfidf

In [107]:
def summarize(para):
    sentences=tb(para).sentences
    
    if len(sentences) is 0:
        return 'nothing';
    
    lower_sentences=[]
    for sentence in sentences:
        lower_sentences.append(sentence.lower())
        
    sent_idfs=get_sent_idfs(lower_sentences)
    
    X1 = np.array(sent_idfs)
    X1=X1.reshape(-1,1)
    X2 = np.zeros((len(X1),1))
    X = np.hstack((X1,X2))
    
    cluster_size=max(1, len(X)//2)
    kmeans = KMeans(n_clusters=cluster_size, random_state=0).fit(X)
    
    counter={}
    n=len(X)
    for i in range(cluster_size):
        counter[i]=0
    for i in range(n):
        counter[kmeans.labels_[i]]+=1
    mx=0
    for i in range(cluster_size):
        if counter[i]>counter[mx]:
            mx=i
            
    summarized_para=""
    for i in range(len(sentences)):
        if kmeans.labels_[i] == mx:
            summarized_para+=(str(sentences[i])+" ")
    
    return summarized_para